In [1]:
# DATA READING

# Install required library (if not already installed)
!pip install pandas openpyxl

import pandas as pd

# File path to your Excel file
file_path = "/content/Sales_English_Translated.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Display the first 5 rows
print(df.head())


      Car Series      Brand  Sales Year  Sales Month Car Series Type  \
0  A5 Wing Dance  southeast        2018           11           Sedan   
1  A5 Wing Dance  southeast        2018           12           Sedan   
2  A5 Wing Dance  southeast        2019            1           Sedan   
3  A5 Wing Dance  southeast        2019            2           Sedan   
4  A5 Wing Dance  southeast        2019            3           Sedan   

       Brand Energy Type     Car Size Brand Origin Country  First Launch Year  \
0  Traditional Car Brand  Compact Car                China               2018   
1  Traditional Car Brand  Compact Car                China               2018   
2  Traditional Car Brand  Compact Car                China               2018   
3  Traditional Car Brand  Compact Car                China               2018   
4  Traditional Car Brand  Compact Car                China               2018   

   Brand Founded Year  Brand Entry to China Year  Sales  
0                1995 

In [3]:
# MERGING THE DATASET
# Install required libraries
!pip install pandas openpyxl

import pandas as pd

# ==============================
# STEP 1: Load your datasets
# ==============================

# Example: Sales dataset (Excel or CSV)
sales_file = "sales_data.xlsx"   # change to your filename
reviews_file = "reviews_data.xlsx"  # change to your filename

# Read datasets
sales_df = pd.read_excel(sales_file)
reviews_df = pd.read_excel(reviews_file)

print("✅ Sales Dataset Preview:")
print(sales_df.head(), "\n")

print("✅ Reviews Dataset Preview:")
print(reviews_df.head(), "\n")

# ==============================
# STEP 2: Inspect Columns
# ==============================
print("Sales dataset columns:", sales_df.columns.tolist())
print("Reviews dataset columns:", reviews_df.columns.tolist())

# ==============================
# STEP 3: Check common columns
# ==============================
# Usually, we merge on product_id, product_name, or some unique identifier
common_columns = set(sales_df.columns).intersection(set(reviews_df.columns))
print("Common columns between datasets:", common_columns)

# ==============================
# STEP 4: Merge datasets
# ==============================

# 4.1 Inner Join (only matching records)
merged_inner = pd.merge(sales_df, reviews_df, on="product_id", how="inner")
print("Inner Join (Only Matching Records):")
print(merged_inner.head(), "\n")

# 4.2 Left Join (all sales + matching reviews)
merged_left = pd.merge(sales_df, reviews_df, on="product_id", how="left")
print("Left Join (All Sales, Matching Reviews if Any):")
print(merged_left.head(), "\n")

# 4.3 Right Join (all reviews + matching sales)
merged_right = pd.merge(sales_df, reviews_df, on="product_id", how="right")
print("Right Join (All Reviews, Matching Sales if Any):")
print(merged_right.head(), "\n")

# 4.4 Outer Join (all records, sales + reviews, fill NaN if no match)
merged_outer = pd.merge(sales_df, reviews_df, on="product_id", how="outer")
print("Outer Join (All Records from Both Datasets):")
print(merged_outer.head(), "\n")

# ==============================
# STEP 5: Advanced Merging
# ==============================

# If datasets don't share a direct column, you can merge on multiple keys
# Example: product_id + date
if "date" in sales_df.columns and "date" in reviews_df.columns:
    merged_multi = pd.merge(sales_df, reviews_df, on=["product_id", "date"], how="inner")
    print("Multi-key Merge (product_id + date):")
    print(merged_multi.head(), "\n")

# ==============================
# STEP 6: Save the merged dataset
# ==============================
output_file = "merged_dataset.xlsx"
merged_outer.to_excel(output_file, index=False)  # saving the widest merged dataset
print(f"✅ Final merged dataset saved as {output_file}")



In [4]:
# DATA TRANSLATION


# Install required libraries
!pip install deep-translator pandas openpyxl tqdm

import pandas as pd
from deep_translator import GoogleTranslator
from google.colab import files
from tqdm import tqdm

# Upload your file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Load the dataset
df = pd.read_csv(file_name)

# Step 1: Extract all unique text values
unique_texts = set()
for col in df.columns:
    unique_texts.update(df[col].astype(str).unique())

# Step 2: Filter only those containing Chinese characters
def contains_chinese(text):
    return any('\u4e00' <= char <= '\u9fff' for char in text)

chinese_texts = [txt for txt in unique_texts if contains_chinese(txt)]

# Step 3: Translate unique Chinese texts
translations = {}
for txt in tqdm(chinese_texts, desc="Translating"):
    try:
        translations[txt] = GoogleTranslator(source='auto', target='en').translate(txt)
    except:
        translations[txt] = txt  # fallback to original if failed

# Step 4: Replace in dataframe
def translate_cell(cell):
    cell_str = str(cell)
    return translations.get(cell_str, cell_str)

translated_df = df.applymap(translate_cell)

# Save translated file
output_file = "Reviews_English_Translated.xlsx"
translated_df.to_excel(output_file, index=False)

# Download translated file
files.download(output_file)